In [0]:

storage_account_name = "netflixprojectdlt"

# List of containers you need to mount
containers = ["bronze", "silver", "gold", "raw" , "metastore"]

# SAS token for authentication (Ensure it's valid)
sas_token = "sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2025-03-15T00:39:21Z&st=2025-03-14T16:39:21Z&spr=https&sig=TTa9xSA0VufPqqFCIbTRwvCilFNBFcIUM9gd1WhhGC8%3D"

# Loop through each container and mount it
for container in containers:
    mount_point = f"/mnt/{container}"  # Define the mount point

    # Unmount if already mounted
    if any(mount.mountPoint == mount_point for mount in dbutils.fs.mounts()):
        dbutils.fs.unmount(mount_point)
        print(f"Unmounted existing mount: {mount_point}")

    # Mount the container
    dbutils.fs.mount(
        source=f"wasbs://{container}@{storage_account_name}.blob.core.windows.net",
        mount_point=mount_point,
        extra_configs={f"fs.azure.sas.{container}.{storage_account_name}.blob.core.windows.net": sas_token}
    )
    print(f"Mounted {container} successfully at {mount_point}")


In [0]:
display(dbutils.fs.mounts())


In [0]:
display(dbutils.fs.ls("/mnt/raw"))



In [0]:
# Define checkpoint location in DBFS instead of abfss
checkpoint_location = "dbfs:/mnt/silver/checkpoints/"

df = spark.readStream \
    .format("cloudFiles") \
    .option("cloudFiles.format", "csv") \
    .option("cloudFiles.schemaLocation", checkpoint_location) \
    .load("dbfs:/mnt/raw/")  # Using mounted path instead of abfss



In [0]:
df.display()


In [0]:
df.writeStream \
    .outputMode("append") \
    .option("checkpointLocation", "dbfs:/mnt/silver/checkpoints/") \
    .start("dbfs:/mnt/bronze/netflix_titles")